In [ ]:
# Unpack and Clean data

In [ ]:
# Imports
import os
import boto3
import zipfile
import shutil
import progressbar

# Notebook specific
from IPython.display import clear_output

# Exploring data
import json
from pathlib import Path

In [ ]:
#Unzip files to new folder
# https://stackoverflow.com/questions/3451111/unzipping-files-in-python
path_to_zip_file = '../data/arxiv.zip'
directory_to_extract_to = '../data/loading/'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
# Lets look at those files using ijson to avoid loading entire files
folder_name = '../data/loading/'
directory = os.fsencode(folder_name)
max_lines_to_load = 2

# https://www.aylakhan.tech/?p=27

def load_data(full_name, lines_to_load = 2):
    with open(full_name, 'r') as f:
        data  = []
        lines = 0
        for line in f: 
            data.append(json.loads(line))
            lines += 1
            if lines >= lines_to_load:
                return data
            
for file in os.listdir(directory):
    file_name = os.fsdecode(file)
    full_name = os.path.join(folder_name, file_name)
    file_size = Path(full_name).stat().st_size / 1024 / 1024 # Mb
    if file_name.endswith(".json") and file_size > 500: # Only open large files 
        print("Filename: ", file_name, "\n")
        data = load_data(full_name, max_lines_to_load)
        for line in data:
            print(line)
        print('\n')

In [ ]:
# Clean data

In [ ]:
# Move files to individual folders
os.makedirs(os.path.dirname('../data/loading/metadata/'), exist_ok=True)
os.makedirs(os.path.dirname('../data/loading/authors/'), exist_ok=True)
os.makedirs(os.path.dirname('../data/loading/citations/'), exist_ok=True)
os.makedirs(os.path.dirname('../data/loading/classifications/'), exist_ok=True)

In [ ]:
# Copy the data to loading folder to group it, which lets us sync the folder, and we can delete it after to save space
src = '../data/loading/arxiv-metadata-oai-snapshot.json'
dst = '../data/loading/metadata/arxiv-metadata-oai-snapshot.json'
shutil.move(src, dst)

src = '../data/loading/authors-parsed.json'
dst = '../data/loading/authors/authors-parsed.json'
shutil.move(src, dst)

src = '../data/loading/internal-citations.json'
dst = '../data/loading/citations/internal-citations.json'
shutil.move(src, dst)

src = '../data/subject-classifications.csv'
dst = '../data/loading/classifications/subject-classifications.csv'
shutil.move(src, dst)

In [ ]:
# Create sample for metadata
# Run in bash: head -100000 arxiv-metadata-oai-snapshot.json > arxiv-metadata-oai-snapshot-sample.json